In [0]:
import numpy as np
from scipy.misc import imread, imresize
import tensorflow as tf
!pip3 install -q tflearn
import tflearn
from glob import glob # helps find files in a folder
import os # to navigate through filesystem
from google.colab import drive
import matplotlib.pyplot as plt # need to plot images
import os , sys
import glob
from scipy.misc import imread, imresize, imshow
from tflearn.layers.core import input_data
from tflearn.layers.conv import global_avg_pool, conv_2d, max_pool_2d
from tflearn.layers.core import fully_connected
from tflearn.layers.estimator import regression
from tflearn.activations import relu
from tflearn.layers.normalization import batch_normalization as bn

In [2]:
drive.mount('/content/drive') # mounts drive to colaboratory

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
os.listdir('/content/drive/My Drive/DeepLearningFall2018/')

['Colab_Intro.ipynb',
 'DeepLearningFall2018-AtaniaPaul',
 'Multilayer_perceptron_AtaniaPaul.ipynb',
 'Colab Notebooks',
 'Convolutional Neural Networks Lecture 2 (1).pptx',
 'Atania Paul_DeepLearning_HW3.gdoc',
 'iris_flower_data.csv',
 'Midterm_Exam_AtaniaPaul.ipynb',
 'CNN_AtaniaPaul.ipynb',
 'Data',
 'mnist',
 'GAN.ipynb',
 'Final Project.gslides',
 'Load_Data_Example.ipynb',
 'ROC_ Example.ipynb',
 'CNN_HW4_AtaniaPaul.ipynb',
 'residual_block.py',
 'Fruit_data.h5',
 'Graph',
 'Final_Project_AtaniaPaul.ipynb']

In [0]:
os.chdir('/content/drive/My Drive/DeepLearningFall2018/')

In [5]:
os.listdir()

['Colab_Intro.ipynb',
 'DeepLearningFall2018-AtaniaPaul',
 'Multilayer_perceptron_AtaniaPaul.ipynb',
 'Colab Notebooks',
 'Convolutional Neural Networks Lecture 2 (1).pptx',
 'Atania Paul_DeepLearning_HW3.gdoc',
 'iris_flower_data.csv',
 'Midterm_Exam_AtaniaPaul.ipynb',
 'CNN_AtaniaPaul.ipynb',
 'Data',
 'mnist',
 'GAN.ipynb',
 'Final Project.gslides',
 'Load_Data_Example.ipynb',
 'ROC_ Example.ipynb',
 'CNN_HW4_AtaniaPaul.ipynb',
 'residual_block.py',
 'Fruit_data.h5',
 'Graph',
 'Final_Project_AtaniaPaul.ipynb']

In [0]:

# create a 
def residual_block(incoming, n_filters, filter_size, n_blocks, stride=1):
  
  for block in range(n_blocks):
    n, h, w, c = incoming.get_shape().as_list() # get shape of input
    
    if block > 0 and stride > 1:
      stride = 1
  
   # convolution 1
    conv1 = conv_2d(incoming, n_filters, filter_size, activation='linear', strides=stride)
    conv1_norm = relu(bn(conv1))
  
   # convolution 2
    conv2 = conv_2d(conv1_norm, n_filters, filter_size, activation='linear', strides=1)
    conv2_norm = bn(conv2)
  
    if stride > 1:
      incoming = max_pool_2d(incoming, 3, stride)
  
    if c != n_filters:
      incoming = conv_2d(incoming, n_filters, 1, activation='linear', strides=1, bias=False)
      
   # elementwise addition between conv2_norm and input
    incoming = incoming + conv2_norm
  
   # relue on addition
    incoming = relu(incoming)
    
  return incoming


In [0]:
def img_reader(dir, imsz, file_ex=None, sort=False):
    ''' A function that reads in images contained
        in each folder in dir, resizes them to
        imsz[0] x imsz[1], and attaches a label to
        each image based on the folder it came from.

        Args:
             dir: directory where the image folders are
                  located.
             imsz: a list of length 2, where each number
                   represents the height and width each
                   image will be resized to. If all images
                   are the same size and don't need to be
                   resized, just put the size they already
                   are.
              file_ex: if the images you want to compile
                       are all of the same extension and/or
                       there are other file types in the
                       folders that you don't want to read,
                       you can include the extension as file_ex.
                       Example, file_ex = '.jpg'
    '''
    
    main_dir = os.getcwd()
    
    if file_ex is None:
        file_ex = '*'
    else:
        file_ex = '*' + file_ex

    folders = os.listdir(dir)
    os.chdir(dir)
    num_ims=sum([len(files) for r, d, files in os.walk(os.getcwd())])
    imgs = np.zeros([num_ims, imsz[0], imsz[1], 3])
    labels = np.zeros([num_ims, 1])

    for subdir, folder in enumerate(folders):
        os.chdir(dir + '/' + folder)
        files = glob.glob(file_ex)
        if sort:
            files.sort()

        for i, file in enumerate(files):
            if os.path.isfile(file):
                img = imread(file)
                img = imresize(img, [imsz[0], imsz[1]])
                imgs[i, ...] = img
                labels[i, 0] = subdir

    os.chdir(main_dir)            
    return imgs, labels

In [8]:
os.getcwd() # see what folder you are in

'/content/drive/My Drive/DeepLearningFall2018'

In [0]:
os.chdir('Data')

In [10]:
X, Y = img_reader(os.getcwd(), [150, 150])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


KeyboardInterrupt: ignored

In [0]:
import h5py
h5file = h5py.File('Fruit_data.h5', 'a')
h5file.create_dataset('X', data=X)
h5file.create_dataset('Y', data=Y)
h5file.close()

In [11]:
os.getcwd()

'/content/drive/My Drive/DeepLearningFall2018/Data/bananas'

In [0]:
os.chdir('..')

In [0]:
import h5py
h5file = h5py.File('Fruit_data.h5', 'r')
X = np.asarray(h5file['X'])
Y = np.asarray(h5file['Y'])

In [0]:
from tflearn.data_utils import to_categorical
Y = to_categorical(Y, 6)

In [18]:
print(X.shape)

(1992, 150, 150, 3)


In [0]:
tf.reset_default_graph()

In [26]:
!pip3 install tensorboardcolab

from tensorboardcolab import *

tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://dd415f43.ngrok.io


In [0]:
# create the input layer for the network
input_layer = input_data([None, 150, 150, 3])

# first convolution --- 75 x 75 x 64
conv1 = conv_2d(input_layer, 64, 7, strides=2, activation='relu')

# max pooling --- 37 x 37 x 64
pool1 = max_pool_2d(conv1, 3, 2)

# residual blocks 1-3 --- 37 x 37 x 64
residual_block1 = residual_block(pool1, # input to this layer
                                 64, # number of features to look for
                                 3, # filter size
                                 3) # number of residual blocks

# residual blocks 4-7 --- 18 x 18 x 128
residual_block2 = residual_block(residual_block1, # input 
                                 128, # num. features to look for
                                 3, # filter size
                                 4, # number of blocks
                                 2) # stride length 

# residual blocks 8-13 --- 9 x 9 x 256
residual_block3 = residual_block(residual_block2,
                                 256,
                                 3,
                                 2, 
                                 2)

# residual blocks 14-16 --- 4 x 4 x 512
residual_block4 = residual_block(residual_block3,
                                 512,
                                 3,
                                 2,
                                 2)

# global average pooling --- 512
gap = global_avg_pool(residual_block4)

# output layer
output_layer = fully_connected(gap, 6, activation='softmax')

In [0]:
# describe optimization 
optimizer = regression(output_layer, optimizer='adam', learning_rate=0.00001)

In [0]:
# build the model
model = tflearn.DNN(optimizer, tensorboard_verbose=3, tensorboard_dir='./Graph')

In [0]:
# training
model.fit(X,
          Y,
          batch_size=32,
          n_epoch=100,
          validation_set=0.2,
          show_metric=True,
          snapshot_step=100)

Training Step: 876  | total loss: 7.75860 | time: 254.688s
| Adam | epoch: 018 | loss: 7.75860 - acc: 0.3882 -- iter: 0832/1593
